In [3]:
from pathlib import Path
from PIL import Image, ImageDraw
import json

def split_battle_spot(file_name: str):
  # Split the battle spots (user and opponents battle spot)
  path_name = Path(file_name)
  battle_spot = Image.open(file_name)
  crop_offset = 0
  field_name = path_name.stem
  if field_name == "cynthia":
    crop_offset = 1
  elif field_name in ["aaron", "bertha", "flint", "lucian"]:
    crop_offset = 2

  battle_spot_left = battle_spot.crop((0, battle_spot.height / 2 + (crop_offset), 256, battle_spot.height))
  battle_spot_right = battle_spot.crop((0, 0, 256, battle_spot.height / 2 + (crop_offset)))
  
  return battle_spot_left, battle_spot_right

def generate_shadow(img: Image, center: tuple[int, int], widths: list[int], doubled_center: bool):
    """Draws an ellipsis shadow on image."""
    img_draw = ImageDraw.Draw(img)
    
    for i, w in enumerate(widths):
        x1 = center[0] - (w / 2)
        x2 = center[0] + (w / 2)
        y1 = center[1] - i if doubled_center == False else center[1] - i - 1
        y2 = center[1] + i
        img_draw.line(xy=[(x1, y1), (x2, y1)], fill=(98, 85, 101, 255), width=1)
        img_draw.line(xy=[(x1, y2), (x2, y2)], fill=(98, 85, 101, 255), width=1)
    
    return img

def generate_encounter(sprite_file: str, field_type: str, battle_spot_type: str, pkmn_index: int, x: int, y: int, center: tuple[int, int], widths: list[int], doubled_center: bool):
  """Generates an recreation of a Pokemon Encounter"""
  dataset_dir = "../assets/PokemonGen4/"
  rand_field = Image.open(f"../assets/fields/{field_type}.png")
  #rand_spot = Image.open("battle_spots/grass_day.png")
  rand_spot_left, rand_spot_right = split_battle_spot(f"../assets/battle_spots/{battle_spot_type}.png")
  sprite = Image.open(sprite_file)
  sprite = sprite.convert("RGBA")

  rand_field.paste(im=rand_spot_left, box=(-64, 120), mask=rand_spot_left)
  rand_field.paste(im=rand_spot_right, box=(64, 64), mask=rand_spot_right)
  rand_field = generate_shadow(img=rand_field, center=center, widths=widths, doubled_center=doubled_center)
  #img_draw = ImageDraw.Draw(rand_field)
  #img_draw.line(xy=[(5, 5), (5, 100)], fill=(98, 85, 101, 255), width = 5)
  rand_field.paste(im=sprite, box=(x, y), mask=sprite)
  return rand_field

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

os.makedirs("../dataset", exist_ok=True)

with open("encounter_info.json") as f:
    coordinates_dict = json.load(f)

with open("encounter_shadows.json") as f:
    shadow_dict = json.load(f)

with open("field_map.json") as f:
    field_map = json.load(f)

# Generate field & battle spot pairs
f_bs_pairs = []
weights = [] # Custom weights set to prioritize field and battle spots likely to encounter in the wild

for key in field_map.keys():
    for bs in field_map[key]["battle_spots"]:
        f_bs_pairs.append((key, bs))
        weights.append(field_map[key]["weight"])

weights = np.array(weights, dtype=float)
weights /= weights.sum()

rng = np.random.default_rng()
games = ["dp", "pt", "hgss"]
for i in range(493): # Pokemon 1 - 493
    pkmn_index = i + 1 
    os.makedirs(f"../dataset/{pkmn_index}", exist_ok=True)
    for j in range(2): # Generate images for normal, then shiny
        sprite_type = "n" if j == 0 else "s"
        for game in games:
            sprite_files = list(Path(f"../assets/PokemonGen4/sprites_db").glob(f"{str(pkmn_index)}/{game}-{sprite_type}.png"))
            sprite_files += list(Path(f"../assets/PokemonGen4/sprites_db").glob(f"{str(pkmn_index)}-*/{game}-{sprite_type}.png"))
            #print(sprite_files)
            os.makedirs(f"../dataset/{pkmn_index}/{sprite_type}", exist_ok=True)
            for k in range(len(f_bs_pairs)):
                field_type, battle_spot = f_bs_pairs[k]
                #print(field_type, battle_spot)
                sprite_file = sprite_files[k % len(sprite_files)]
                idx = sprite_file.parent.stem.split("-")[0]
                x, y = coordinates_dict[idx][game]["x"], coordinates_dict[idx][game]["y"]
                center = shadow_dict[idx][game]["center"]
                widths = shadow_dict[idx][game]["widths"]
                dc = shadow_dict[idx][game]["doubled_center"]
                encounter_img = generate_encounter(
                        sprite_file=sprite_file, 
                        field_type=field_type,
                        battle_spot_type=battle_spot,
                        pkmn_index = pkmn_index,
                        x=x,
                        y=y,
                        center=center,
                        widths=widths,
                        doubled_center=dc
                )
                field_type_shortened = ''.join([x[0] for x in field_type.split("_")])
                battle_spot_shortened = ''.join([x[0] for x in battle_spot.split("_")])
                #print(field_type_shortened, battle_spot_shortened)
                encounter_img.save(f"../dataset/{pkmn_index}/{sprite_type}/{game}_{field_type_shortened}_{battle_spot_shortened}.png")